In [ ]:
# import os
import pandas as pd
# import numpy as np
# import ast
import pandas as pd
pd.set_option('display.max_columns', None)
# import torch
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb
import wandb

In [ ]:
# Import preprocessing module
from preprocessing_pipeline import pipeline_features
# Import W&B module for Sklearn
# from models.training_utils import SklearnTrainer


#### Try W&B config

In [ ]:
# Import wandb and run a test to see ig it works
wandb_project = "Node_Power_Consumption"
wandb.init(project=wandb_project, name="test-run")
wandb.log({"metric": 1})
wandb.finish()

metric,▁
metric,1


### Prepare the dataset for XGBoost

In [61]:
# Read the Marconi 100 data (job table)
df_raw = pd.read_parquet("job_table.parquet")

In [62]:
# df_raw.columns

##### Dataset with node_consumption as target

We choose linear= False as we are trying a XGBoost regression.<br>
Use include_cpu=include_mem= False, we leave as target only node consumption stats: min, mean, max.<br>
Use two groups: main, other.<br>
Leave num_tasks_option= 1 : imputes NAs, leaves the incorrect values of num_tasks as such. <br>

In [63]:
df= pipeline_features(df_raw, linear=False, include_cpu=False, include_mem=False, group_option=2, num_tasks_option=1)

Option leave two groups: main, other.
Option Impute NAs, leave incorrect num_tasks entries.
Option generate DS for tree models.


In [64]:
df.columns

Index(['group', 'num_tasks_final', 'num_tasks_missing_or_inconsistent',
       'submit_time', 'time_limit', 'time_limit_cat', 'num_nodes_req',
       'has_req_nodes', 'num_cores_req', 'cores_per_task', 'num_gpus_req',
       'mem_req', 'has_req_threads_per_core', 'is_shared_job', 'dow_0',
       'dow_1', 'dow_2', 'dow_3', 'dow_4', 'dow_5', 'dow_6', 'dom_1', 'dom_2',
       'dom_3', 'dom_4', 'dom_5', 'dom_6', 'dom_7', 'dom_8', 'dom_9', 'dom_10',
       'dom_11', 'dom_12', 'dom_13', 'dom_14', 'dom_15', 'dom_16', 'dom_17',
       'dom_18', 'dom_19', 'dom_20', 'dom_21', 'dom_22', 'dom_23', 'dom_24',
       'dom_25', 'dom_26', 'dom_27', 'dom_28', 'dom_29', 'dom_30', 'dom_31',
       'hour_0', 'hour_1', 'hour_2', 'hour_3', 'hour_4', 'hour_5', 'hour_6',
       'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12',
       'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18',
       'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23', 'node_power_min',
       'node_power_m

In [65]:
df["dow"] = df["submit_time"].dt.dayofweek 
df["dom"] = df["submit_time"].dt.day                  # 1–31
df["hour"] = df["submit_time"].dt.hour
df["is_weekend"] = df["dow"].isin([5,6]).astype(int)
df["month"] = df["submit_time"].dt.month
df["is_night"] = ((df["hour"] < 7) | (df["hour"] >= 22)).astype(int)
df["is_peak"] = df["hour"].between(9, 18).astype(int)

In [66]:
df.drop(columns=["submit_time", ], inplace=True)

In [67]:
cols_to_drop = (
    df.filter(like='dow_').columns.tolist() +
    df.filter(like='dom_').columns.tolist() +
    df.filter(like='hour_').columns.tolist()
)
df = df.drop(columns=cols_to_drop)

In [68]:
df.columns

Index(['group', 'num_tasks_final', 'num_tasks_missing_or_inconsistent',
       'time_limit', 'time_limit_cat', 'num_nodes_req', 'has_req_nodes',
       'num_cores_req', 'cores_per_task', 'num_gpus_req', 'mem_req',
       'has_req_threads_per_core', 'is_shared_job', 'node_power_min',
       'node_power_mean', 'node_power_max', 'dow', 'dom', 'hour', 'is_weekend',
       'month', 'is_night', 'is_peak'],
      dtype='object')

In [69]:
# Convert categorical columns to category codes
categorical_cols = ["group", "time_limit_cat"]

for col in categorical_cols:
    df[col] = df[col].astype("category").cat.codes

In [70]:
df.head()

,group,num_tasks_final,num_tasks_missing_or_inconsistent,time_limit,time_limit_cat,num_nodes_req,has_req_nodes,num_cores_req,cores_per_task,num_gpus_req,mem_req,has_req_threads_per_core,is_shared_job,node_power_min,node_power_mean,node_power_max,dow,dom,hour,is_weekend,month,is_night,is_peak
0,0,64,False,270,1,16,0,256,4,64,475,0,0,7440,8318.888889,8500,6,31,22,1,5,1,0
1,0,64,False,270,1,16,0,256,4,64,475,0,0,5820,8164.827586,8530,6,31,22,1,5,1,0
2,0,64,False,270,1,16,0,256,4,64,475,0,0,5800,8193.111111,8510,6,31,22,1,5,1,0
3,0,0,True,30,0,1,0,32,32,4,59,0,1,520,630.377358,860,6,31,23,1,5,1,0
4,0,0,True,30,0,1,0,32,32,4,59,0,1,860,860.208333,870,6,31,23,1,5,1,0


In [71]:
# Separate features and target columns
target_cols = [
    "node_power_min",
    "node_power_mean",
    "node_power_max"
]

feature_cols = [
    'group',
    'num_tasks_final',
    'num_tasks_missing_or_inconsistent',
    'time_limit',
    'time_limit_cat',
    'num_nodes_req',
    'has_req_nodes',
    'num_cores_req',
    'cores_per_task',
    'num_gpus_req',
    'mem_req',
    'has_req_threads_per_core',
    'is_shared_job',
    'dow', 'dom', 'hour', 'is_weekend', 'month', 'is_night', 'is_peak'
]

In [72]:
# Check that the number of feature and target columns matches the total number of columns in the dataframe
print(len(feature_cols)+len(target_cols))
print(len(df.columns))

23
23


In [73]:
# Split df into X and Y
X = df[feature_cols].copy()
Y = df[target_cols].copy()

In [74]:
X.head()

,group,num_tasks_final,num_tasks_missing_or_inconsistent,time_limit,time_limit_cat,num_nodes_req,has_req_nodes,num_cores_req,cores_per_task,num_gpus_req,mem_req,has_req_threads_per_core,is_shared_job,dow,dom,hour,is_weekend,month,is_night,is_peak
0,0,64,False,270,1,16,0,256,4,64,475,0,0,6,31,22,1,5,1,0
1,0,64,False,270,1,16,0,256,4,64,475,0,0,6,31,22,1,5,1,0
2,0,64,False,270,1,16,0,256,4,64,475,0,0,6,31,22,1,5,1,0
3,0,0,True,30,0,1,0,32,32,4,59,0,1,6,31,23,1,5,1,0
4,0,0,True,30,0,1,0,32,32,4,59,0,1,6,31,23,1,5,1,0


In [75]:
Y.head()

,node_power_min,node_power_mean,node_power_max
0,7440,8318.888889,8500
1,5820,8164.827586,8530
2,5800,8193.111111,8510
3,520,630.377358,860
4,860,860.208333,870


In [42]:
X.to_csv("X_features.csv", index=False)
Y.to_csv("Y_targets.csv", index=False)

#### Split tr, tt

In [82]:
# Split the data into training, validation, and test sets
# Choose: 70% training, 15% validation, 15% test
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(
    X, Y, test_size=0.30, random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42
)


In [ ]:
# Save the split sets (tr/val/tt) to pickle files
import pickle

X_train.to_parquet("data/X_train.parquet")
X_val.to_parquet("data/X_val.parquet")
X_test.to_parquet("data/X_test.parquet")

y_train.to_parquet("data/y_train.parquet")
y_val.to_parquet("data/y_val.parquet")
y_test.to_parquet("data/y_test.parquet")

#### Functions to integrate W&B

The functions are in file: train_xgb_sweep.py

The sweep is performed directly from the Anaconda terminal. <br>
Activate the right environment and go to the folder that contains the sweep.yaml file.<br>
Run: <br>
">"wandb sweep sweep_n.yaml <br>
where n is the number of sweep. <br>
Then, activate the agent with the sweep_id given in the previous step: <br>
>wand agent <entity/project_name/sweep_id> <br>
Example: <br>
>wandb agent 100496657-universidad-carlos-iii-de-madrid/DS-HPE/s2xeaxuh <br>


## Legacy

#### Evaluate the best model over test set

In [ ]:
from data import X_train, y_train, X_val, y_val, X_test, y_test
import numpy as np

# Prepare data
X_train_full = pd.concat([X_train, X_val], axis=0)
y_train_full = pd.concat([y_train, y_val], axis=0)

X_train_full_np = X_train_full.to_numpy()
y_train_full_np = y_train_full.to_numpy()
X_test_np = X_test.to_numpy()
y_test_np = y_test.to_numpy()

# Build model with chosen best hyperparameters
best_model = MultiOutputRegressor(
    xgb.XGBRegressor(
        tree_method="hist",
        enable_categorical=True,
        n_estimators=BEST_N_ESTIMATORS,
        max_depth=BEST_MAX_DEPTH,
        learning_rate=BEST_LR,
        subsample=BEST_SUBSAMPLE,
        colsample_bytree=BEST_COLSAMPLE,
        min_child_weight=BEST_MIN_CHILD_WEIGHT,
        random_state=42,
    )
)

best_model.fit(X_train_full_np, y_train_full_np)

# Final test evaluation
metrics_test, y_pred_test = evaluate_model(best_model, X_test_np, y_test_np, model_type="sklearn")
print(metrics_test)


In [41]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

def evaluate(model, X, y, name):
    y_pred = model.predict(X)
    rmse = np.sqrt(mean_squared_error(y, y_pred))
    r2 = r2_score(y, y_pred)
    print(f"{name} RMSE: {rmse:.3f}")
    print(f"{name} R²:   {r2:.3f}")

evaluate(model, X_val, y_val, "VAL")
evaluate(model, X_test, y_test, "TEST")

VAL RMSE: 2262.667
VAL R²:   0.950
TEST RMSE: 2044.847
TEST R²:   0.957


In [ ]:
### FOR CLASSIC MODELS

# Assuming you have your data in X (features) and y (labels)
"""X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.7, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)  # Fit only on train!
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

"""
### FOR TORCH MODELS

# Convert to tensors (on CPU)
"""X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)  # Use float32 for regression

X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Create datasets and loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 1028
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)"""

In [27]:
print(f"Number of batches - Train: {len(train_loader)}, Val: {len(val_loader)}, Test: {len(test_loader)}")

Number of batches - Train: 68, Val: 79, Test: 79


In [28]:
import sys
sys.path.append('./models/code_models/')

import sklearn_models
import torch_models
import training_utils

## Legacy: unused code in this notebook

### SKLEARN 

In [18]:
rf_model = sklearn_models.get_random_forest()

rf_trainer = training_utils.SklearnTrainer(
    model=rf_model,
    model_name="RandomForest_Default",
    project_name="Test",
    entity="iqbalch-universidad-carlos-iii-de-madrid" 
)

rf_model, rf_metrics = rf_trainer.train(
    X_train_scaled, y_train,
    X_val_scaled, y_val,
    config= rf_model.get_params()
)

print("\nValidation Metrics:")
for key, value in rf_metrics.items():
    if 'val' in key:
        print(f"{key}: {value:.4f}")

test_metrics, test_predictions = training_utils.evaluate_model(
    rf_model, X_test_scaled, y_test, model_type="sklearn"
)

print("\nTest Set Metrics:")
for key, value in test_metrics.items():
    print(f"{key}: {value:.4f}")


wandb: Currently logged in as: iqbalch (iqbalch-universidad-carlos-iii-de-madrid) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Training RandomForest_Default...


train_mae_max_power,▁
train_mae_mean,▁
train_mae_mean_power,▁
train_mae_min_power,▁
train_r2_max_power,▁
train_r2_mean,▁
train_r2_mean_power,▁
train_r2_min_power,▁
train_rmse_max_power,▁
train_rmse_mean,▁
+14,...



Validation Metrics:
val_rmse_mean_power: 161.7926
val_mae_mean_power: 113.5575
val_r2_mean_power: 0.3041
val_rmse_min_power: 159.8107
val_mae_min_power: 108.7766
val_r2_min_power: 0.3294
val_rmse_max_power: 221.0320
val_mae_max_power: 156.4954
val_r2_max_power: 0.3231
val_rmse_mean: 180.8785
val_mae_mean: 126.2765
val_r2_mean: 0.3189

Test Set Metrics:
test_rmse_mean_power: 161.3757
test_mae_mean_power: 113.4599
test_r2_mean_power: 0.3050
test_mape_mean_power: 15.8715
test_rmse_min_power: 160.0210
test_mae_min_power: 109.1074
test_r2_min_power: 0.3190
test_mape_min_power: 22.8296
test_rmse_max_power: 221.0219
test_mae_max_power: 156.7295
test_r2_max_power: 0.3255
test_mape_max_power: 19.9669
test_rmse_mean: 180.8062
test_mae_mean: 126.4323
test_r2_mean: 0.3165
test_mape_mean: 19.5560


### TORCH

In [ ]:
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from torch.utils.data import TensorDataset, DataLoader

In [31]:
input_dim = X_train_scaled.shape[1]
siple_mlp_model = torch_models.SimpleMLP(input_dim=input_dim)

resnet_trainer = training_utils.PyTorchTrainer(
    model=siple_mlp_model,
    model_name="MLP Model",
    project_name="Test",
    entity="iqbalch-universidad-carlos-iii-de-madrid"
)

resnet_model, resnet_best_metrics = resnet_trainer.train(
    train_loader=train_loader,
    val_loader=val_loader,
    epochs=150,
    lr=0.001,
    weight_decay=1e-5,
    patience=15,
)

# Evaluate on test
resnet_test_metrics, _ = training_utils.evaluate_model(
    resnet_model, X_test_scaled, y_test, model_type="pytorch"
)

print("\nTest Set Metrics:")
print(f"RMSE: {resnet_test_metrics['test_rmse_mean']:.4f}")
print(f"MAE: {resnet_test_metrics['test_mae_mean']:.4f}")
print(f"R2: {resnet_test_metrics['test_r2_mean']:.4f}")

c:\Users\iqbal\anaconda3\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Early stopping at epoch 127


epoch,▁▁▂▂▂▂▂▃▃▃▄▄▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇█
lr,██████████████████████████▄▄▃▃▃▃▃▃▃▂▂▂▂▁
train_loss,███▇▆▄▄▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▇▆▆▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mae_max_power,███▇▆▅▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mae_mean_power,██▇▇▇▆▅▄▄▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mae_min_power,███▆▆▅▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_r2_max_power,▁▂▃▃▄▄▅▅▅▆▆▇▇▇██████████████████████████
val_r2_mean_power,▁▂▂▄▄▆▆▇▇▇██████████████████████████████
+5,...



Test Set Metrics:
RMSE: 211.1253
MAE: 156.9213
R2: 0.0703


### Legacy

In [ ]:
# CONSTANTS

# data_path = r"./data/interm/data_to_train_meantime.csv"

In [ ]:
# LOAD THE ESSENTIALS

# df = pd.read_csv(data_path)